In [ ]:
import pandas as pd
df_dev = pd.read_csv("data/development.csv")
df_eva = pd.read_csv("data/evaluation.csv")
df_merged = pd.merge(df_dev, df_eva, sort=False)

Keep only negpmax & pmax

In [ ]:
columns_to_remove = []

for i in [0, 7, 12, 15, 16, 17]:
    columns_to_remove.extend([f"pmax[{i}]", f"negpmax[{i}]"])
for i in range(0,18):
    columns_to_remove.extend([f"area[{i}]",f"tmax[{i}]", f"rms[{i}]"])
columns_to_remove.extend(["Id"])
df_merged_dropped = df_merged.drop(columns=columns_to_remove)
df_merged_dropped.columns

In [ ]:
train_valid_mask = ~df_merged_dropped["x"].isna()
feature_names = df_merged_dropped[train_valid_mask].drop(columns=["x", "y"]).columns
X = df_merged_dropped.drop(columns=["x", "y"])
y = df_merged_dropped[["x", "y"]].values
X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]

GRID SEARCH

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics.pairwise import paired_distances

#params = {
#    "max_depth": [26,27,28,29],
#    "n_estimators":[300,400,500,600,700],
#    "random_state":[42],
#    "max_features":[None,"sqrt"],
#    "min_samples_split":[2,3,4],
#    "criterion":["friedman_mse","mse"]
#}
#
#for config in ParameterGrid(params):
#    print("fit started with conf: ", config)
#    regr_multirf = MultiOutputRegressor(ExtraTreesRegressor(**config))
#    regr_multirf.fit(X_train, y_train)
#    y_pred = regr_multirf.predict(X_valid)
#    distance = paired_distances(y_pred,y_valid).mean()
#    print("fit ended with: ", distance)

Best conf: 29, 700, 42, None, 2, friedman_mse

In [ ]:
regr_multirf = MultiOutputRegressor(
    ExtraTreesRegressor(n_estimators=700, max_depth=29, random_state=42, criterion="friedman_mse")
)
regr_multirf.fit(X_train_valid, y_train_valid)

Predict

In [ ]:
y_test_pred = regr_multirf.predict(X_test)
y_eva_pred = pd.DataFrame(y_test_pred, columns=['Colonna1', 'Colonna2'])
y_eva_pred['indice'] = range(len(y_eva_pred))
y_eva_pred['Concatenato'] = y_eva_pred.apply(lambda row: '|'.join(map(str, row[['Colonna1', 'Colonna2']])), axis=1)
y_eva_pred = y_eva_pred.drop(['Colonna1', 'Colonna2', 'indice'], axis=1)
y_eva_pred.to_csv("output.csv", index_label="Id", header=['Predicted'])